# Getting Started with *earthaccess*
### Analyzing Sea Level Rise

## Overview
This tutorial analyzes global sea level rise from satellite altimetry data and compares it to a historic record. We will be reproducing the plot from this infographic: [ NASA-led Study Reveals the Causes of Sea Level Rise Since 1900](https://grace.jpl.nasa.gov/news/113/nasa-led-study-reveals-the-causes-of-sea-level-rise-since-1900/).

## Learning Objectives
- Search for data programmatically using keywords or datasets' `concept_id`
- Access data using the `earthaccess` python library
- Visualize sea level rise trends from altimetry datasets and compare against historic records

## Requirements
    
### 1. Compute environment
* This notebook can run anywhere thanks to `earthaccess`!
    
### 2. Earthdata Login
An Earthdata Login account is required to access data, as well as discover restricted data, from the NASA Earthdata system. Thus, to access NASA data, you need Earthdata Login. Please visit https://urs.earthdata.nasa.gov to register and manage your Earthdata Login account. This account is free to create and only takes a moment to set up.

### Import (or Install) Packages

In [ ]:
import logging
logging.basicConfig(level=logging.INFO,
                    force = True)

try:
    import earthaccess
    import xarray as xr
    from pyproj import Geod
    import numpy as np
    import hvplot.xarray
    from matplotlib import pyplot as plt
    from pprint import pprint
    import panel as pn
    import panel.widgets as pnw
    from pqdm.threads import pqdm
except ImportError as e:
    logging.warning("installing missing dependencies... ")
    # %pip install -q earthaccess matplotlib hvplot pyproj xarray numpy h5netcdf panel pqdm
finally:
    import earthaccess
    import xarray as xr
    from pyproj import Geod
    import numpy as np
    import hvplot.xarray
    from matplotlib import pyplot as plt
    from pprint import pprint
    import panel.widgets as pnw
    from pqdm.threads import pqdm
    logging.info("Dependencies imported")


INFO:root:Dependencies imported


## earthaccess and NASA's EDL

We recommend authenticating your Earthdata Login (EDL) information using the `earthaccess` python package as follows:

In [3]:
auth = earthaccess.login()

INFO:earthaccess.auth:You're now authenticated with NASA Earthdata Login


## Search for Sea Surface Height Data

Let's find the first four collections that match a keyword search for `Sea Surface Height` and print out the first two.

In [4]:
import warnings
warnings.filterwarnings('ignore')

max_results = 10

collections = earthaccess.search_datasets(
    keyword = "SEA SURFACE HEIGHT",
    cloud_hosted = True,
    count = max_results
)

# Let's print 2 collections
for collection in collections[0:2]:
    # pprint(collection.summary())
    print(pprint(collection.summary()), collection.abstract(), "\n", collection["umm"]["DOI"], "\n\n")

INFO:earthaccess.api:Datasets found: 378


{'cloud-info': {'Region': 'us-west-2',
                'S3BucketAndObjectPrefixNames': ['podaac-swot-ops-cumulus-protected/SWOT_L2_LR_SSH_2.0/',
                                                 'podaac-swot-ops-cumulus-public/SWOT_L2_LR_SSH_2.0/'],
                'S3CredentialsAPIDocumentationURL': 'https://archive.swot.podaac.earthdata.nasa.gov/s3credentialsREADME',
                'S3CredentialsAPIEndpoint': 'https://archive.swot.podaac.earthdata.nasa.gov/s3credentials'},
 'concept-id': 'C2799438306-POCLOUD',
 'file-type': "[{'FormatType': 'Native', 'Format': 'netCDF-4'}]",
 'get-data': ['https://cmr.earthdata.nasa.gov/virtual-directory/collections/C2799438306-POCLOUD',
              'https://search.earthdata.nasa.gov/search/granules?p=C2799438306-POCLOUD'],
 'short-name': 'SWOT_L2_LR_SSH_2.0',
 'version': '2.0'}
None The SWOT Level 2 KaRIn Low Rate Sea Surface Height Data Product from the Surface Water Ocean Topography (SWOT) mission provides global sea surface height and significa

## Access Data

The first dataset looks like it contains data from many altimetry missions, let's explore a bit more! We will access the first granule of the `SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL2205` collection in the month of May for every year that data is available and open the granules via xarray.

In [5]:
granules = []

for year in range(1990, 2020):
    print(f"Searching for data in {year}")
    granule = earthaccess.search_data(doi="10.5067/SLREF-CDRV3",
                                      temporal= (f"{year}-05", f"{year}-06"),
                                      count = 1)
    if len(granule)>0:
        granules.append(granule[0])
print(f"Total granules: {len(granules)}")

Searching for data in 1990


INFO:earthaccess.api:Granules found: 0


Searching for data in 1991


INFO:earthaccess.api:Granules found: 0


Searching for data in 1992


...

Searching for data in 2018


INFO:earthaccess.api:Granules found: 12


Searching for data in 2019


INFO:earthaccess.api:Granules found: 12


Total granules: 27


In [6]:
%%time

def open_file(file):
    ds = xr.open_dataset(file)
    # we can modify here
    return ds

ds_list = pqdm(earthaccess.open(granules), open_file, n_jobs=8)

ds_SSH = xr.concat(ds_list, dim="Time")
ds_SSH

INFO:earthaccess.store:Opening 27 granules, approx size: 0.24 GB
QUEUEING TASKS | : 100%|██████████| 27/27 [00:00<00:00, 7470.56it/s]
PROCESSING TASKS | : 100%|██████████| 27/27 [00:16<00:00,  1.61it/s]
COLLECTING RESULTS | : 100%|██████████| 27/27 [00:00<00:00, 557863.09it/s]
QUEUEING TASKS | : 100%|██████████| 27/27 [00:00<00:00, 12509.25it/s]
PROCESSING TASKS | :   0%|          | 0/27 [00:00<?, ?it/s]INFO:fsspec:BlockCache fetching block (sync) 0
INFO:fsspec:BlockCache fetching block (sync) 0
INFO:fsspec:BlockCache fetching block (sync) 0
INFO:fsspec:BlockCache fetching block (sync) 0
INFO:fsspec:BlockCache fetching block (sync) 0
INFO:fsspec:BlockCache fetching block (sync) 0
INFO:fsspec:BlockCache fetching block (sync) 0
INFO:fsspec:BlockCache fetching block (sync) 0
INFO:fsspec:BlockCache fetching block (async) 1
INFO:fsspec:BlockCache fetching block (sync) 2
INFO:fsspec:BlockCache fetching block (async) 1
INFO:fsspec:BlockCache fetching block (async) 1
INFO:fsspec:BlockCache fet

CPU times: user 6.53 s, sys: 1.07 s, total: 7.61 s
Wall time: 2min 40s


<xarray.Dataset> Size: 449MB
Dimensions:      (Time: 27, Longitude: 2160, nv: 2, Latitude: 960)
Coordinates:
  * Longitude    (Longitude) float32 9kB 0.08333 0.25 0.4167 ... 359.8 359.9
  * Latitude     (Latitude) float32 4kB -79.92 -79.75 -79.58 ... 79.75 79.92
  * Time         (Time) datetime64[ns] 216B 1993-05-03T12:00:00 ... 2019-05-0...
Dimensions without coordinates: nv
Data variables:
    Lon_bounds   (Time, Longitude, nv) float32 467kB -1.388e-17 0.1667 ... 360.0
    Lat_bounds   (Time, Latitude, nv) float32 207kB -80.0 -79.83 ... 79.83 80.0
    Time_bounds  (Time, nv) datetime64[ns] 432B 1993-04-03T12:00:00 ... 2019-...
    SLA          (Time, Latitude, Longitude) float32 224MB nan nan ... 0.09141
    SLA_ERR      (Time, Latitude, Longitude) float32 224MB nan nan ... 0.03657
Attributes: (12/21)
    Conventions:            CF-1.6
    ncei_template_version:  NCEI_NetCDF_Grid_Template_v2.0
    Institution:            Jet Propulsion Laboratory
    geospatial_lat_min:     -79.916664
    geospatial_lat_max:     79.916664
    geospatial_lon_min:     0.083333336
    ...                     ...
    version_number:         2205
    Data_Pnts_Each_Sat:     {"16": 780190, "1001": 668288}
    source_version:         commit dc95db885c920084614a41849ce5a7d417198ef3
    SLA_Global_MEAN:        -0.027657876081274502
    SLA_Global_STD:         0.0859016072308609
    latency:                final

## Plot Sea Level Anomalies

In [8]:
%%time

time = pnw.Player(name='Time', start=0, end=len(ds_SSH.Time)-1, loop_policy='loop', interval=1000)

ds_SSH.SLA.interactive(loc='bottom', aspect="equal").isel(Time=time).hvplot(cmap="inferno", data_aspect=1)

CPU times: user 36.4 ms, sys: 0 ns, total: 36.4 ms
Wall time: 36 ms


BokehModel(combine_events=True, render_bundle={'docs_json': {'92ed3618-8cb6-4ce5-bb81-16d1dbfdb9ad': {'version…

## Recreate the Sea Level Rise Infographic
First, we define a function that will calculate the area in 1/6 by 1/6 degree boxes in order to calculate the global mean of the SSH later.

In [9]:
def ssl_area(lats):
    """
    Calculate the area associated with a 1/6 by 1/6 degree box at latitude specified in 'lats'.

    Parameter
    ==========
    lats: a list or numpy array of size N the latitudes of interest.

    Return
    =======
    out: Array (N) area values (unit: m^2)
    """
    # Define WGS84 as CRS:
    geod = Geod(ellps='WGS84')
    dx=1/12.0
    # create a lambda function for calculating the perimeters of the boxes
    c_area=lambda lat: geod.polygon_area_perimeter(np.r_[-dx,dx,dx,-dx], lat+np.r_[-dx,-dx,dx,dx])[0]
    out=[]
    for lat in lats:
        out.append(c_area(lat))
    return np.array(out)

Let's use the function on our Sea Surface Height dataset.

In [10]:
# note: they rotated the data in the last release, this operation used to be (1,-1)
ssh_area = ssl_area(ds_SSH.Latitude.data).reshape(-1,1)
print(ssh_area.shape)

(960, 1)


Next, we find and open the historic record dataset also using `earthaccess` and `xarray`.

In [15]:
historic_ts_results = earthaccess.search_data(short_name='JPL_RECON_GMSL')
historic_ts=xr.open_mfdataset(earthaccess.open([historic_ts_results[0]]), engine='h5netcdf')

INFO:earthaccess.api:Granules found: 1
INFO:earthaccess.store:Opening 1 granules, approx size: 0.0 GB
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 3953.16it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.09s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<00:00, 42799.02it/s]
INFO:fsspec:BlockCache fetching block (sync) 0
INFO:fsspec:BlockCache fetching block (async) 1
INFO:fsspec:BlockCache joined background fetch without waiting.


ImportError: chunk manager 'dask' is not available. Please make sure 'dask' is installed and importable.

## Let's Plot!

In [ ]:
%%time

plt.rcParams["figure.figsize"] = (16,4)

fig, axs = plt.subplots()
plt.grid(True)

#function to get the global mean for plotting
def global_mean(SLA, **kwargs):
    dout=((SLA*ssh_area).sum()/(SLA/SLA*ssh_area).sum())*1000
    return dout

result = ds_SSH.SLA.groupby('Time').apply(global_mean)

plt.xlabel('Time (year)',fontsize=16)
plt.ylabel('Global Mean SLA (meter)',fontsize=12)
plt.grid(True)

result.plot(ax=axs, color="orange", marker="o", label='Satellite Record')

historic_ts['global_average_sea_level_change'].plot(ax=axs, label='Historical in-situ record', color="lightblue")

plt.legend()
plt.show()

> This Data Story is based on Jinbo Wang's [Earthdata Webinar tutorial](https://github.com/betolink/the-coding-club/blob/main/notebooks/Earthdata_webinar_20220727.ipynb).